In [ ]:
from scrape_modules import ScrapeIshikawa
import json
import slackweb
json_open = open('important.json', 'r')
important_list = json.load(json_open)

In [ ]:
# データスクレイピング
sps_url = important_list['sps_url']
si = ScrapeIshikawa(sps_url)
driver = si.open_driver(False)
sikyou_url_list = si.scrape_sikyou_url(driver)
sikyou_url_list_new = si.get_sikyou_url_new(sikyou_url_list)
si.scrape_sikyou_all(driver, sikyou_url_list_new)
driver.quit()

In [ ]:
# 魚種列，目方列追加
df_species = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[0])
df_size = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[1])
si.scrape_data.insert(3, '魚種',df_species)
si.scrape_data.insert(4, '目方',df_size)

In [ ]:
# データ結合
si.concat_new_data()

In [ ]:
# データ確認
print(len(si.sps_data))
print(len(si.scrape_data))
print(len(si.sps_data_new))
si.sps_data_new

In [ ]:
# データ保存
si.save_sps()

In [ ]:
# # slackに通知
# content = {
#     "icon_url" : "http://drive.google.com/uc?export=view&id=1bbPewWm7dHriVWqHQ2IJe3uzwbN1-RHE",
#     'username': "JF石川市況",
#     "text": f"JF石川の市況データを更新しました\n{important_list['sps_url']}"
#     }

# webhook_url = important_list["slack_webhook_url"]
# requests.post(webhook_url, data = json.dumps(content))